<a href="https://colab.research.google.com/github/codingiamnot/PMP-2024/blob/main/Lab07/pmp_tema6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pymc as pm
import numpy as np

import arviz as az

k_obs = [10, 15, 20, 5, 25, 12, 18]
n_obs = [100, 120, 150, 90, 200, 110, 130]

with pm.Model() as model:
    p = pm.Beta("p", alpha=2, beta=2, shape=7)
    k = pm.Binomial("k", n=n_obs, p=p, observed=k_obs)

    p_mean = pm.Deterministic("p_mean", p.mean())

    # Eșantionare din distribuția aposteriori
    sample = pm.sample(2000, tune=1000, target_accept=0.95, random_seed=1720357)

summary = az.summary(sample, var_names=["p", "p_mean"], round_to=2)
summary

Output()

Output()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p[0],0.11,0.03,0.06,0.17,0.0,0.0,6460.84,2936.99,1.0
p[1],0.14,0.03,0.08,0.20,0.0,0.0,6015.77,2969.72,1.0
p[2],0.14,0.03,0.09,0.20,0.0,0.0,6097.95,2787.60,1.0
p[3],0.07,0.03,0.03,0.12,0.0,0.0,7197.41,2977.50,1.0
p[4],0.13,0.02,0.09,0.18,0.0,0.0,5917.63,2737.88,1.0
p[5],0.12,0.03,0.07,0.18,0.0,0.0,6764.27,3291.79,1.0
p[6],0.15,0.03,0.09,0.21,0.0,0.0,6660.56,2830.91,1.0
p_mean,0.12,0.01,0.10,0.15,0.0,0.0,6632.39,3112.81,1.0


In [15]:
import pymc as pm
import numpy as np
import pandas as pd

data = pd.read_csv('trafic.csv')

approx_change_points = [(7-4)*60, (8-4)*60, (16-4)*60, (19-4)*60]
print(approx_change_points)

with pm.Model() as model:

  change0 = pm.DiscreteUniform("change0",
                               lower=approx_change_points[0]-10,
                               upper=approx_change_points[0]+10)

  change1 = pm.DiscreteUniform("change1",
                               lower=approx_change_points[1]-10,
                               upper=approx_change_points[1]+10)


  change2 = pm.DiscreteUniform("change2",
                               lower=approx_change_points[2]-10,
                               upper=approx_change_points[2]+10)

  change3 = pm.DiscreteUniform("change3",
                               lower=approx_change_points[3]-10,
                               upper=approx_change_points[3]+10)


  lambdas = pm.Exponential("lambdas", lam=1, shape=5)

  minute = np.arange(len(data['nr']))

  lambdaCombined = pm.math.switch(
        (minute < change0), lambdas[0],
        pm.math.switch(
            (minute < change1), lambdas[1],
            pm.math.switch(
                (minute < change2), lambdas[2],
                pm.math.switch((minute < change3), lambdas[3], lambdas[4])
            )
        )
    )

  obs = pm.Poisson("obs", mu=lambdaCombined, observed=data['nr'])

  sample = pm.sample(2000, tune=1000, target_accept=0.95, random_seed=1720357)


summary = az.summary(sample, var_names=["change0", "change1", "change2",
                                        "change3", "lambdas"], round_to=2)
summary



[180, 240, 720, 900]


Output()

Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
change0,184.75,1.85,179.00,187.00,0.23,0.17,123.87,56.05,1.02
change1,249.63,0.79,249.00,250.00,0.05,0.03,408.93,562.36,1.00
change2,719.85,3.34,712.00,725.00,0.33,0.23,108.49,144.08,1.00
change3,892.90,1.40,891.00,895.00,0.10,0.07,388.11,171.23,1.01
lambdas[0],12.17,0.26,11.66,12.64,0.00,0.00,5818.62,2769.71,1.00
lambdas[1],17.61,0.52,16.58,18.54,0.01,0.01,3488.61,2776.36,1.00
lambdas[2],13.00,0.17,12.68,13.30,0.00,0.00,6207.40,2822.14,1.00
lambdas[3],14.69,0.30,14.15,15.25,0.01,0.00,3445.03,3053.28,1.00
lambdas[4],10.14,0.18,9.79,10.49,0.00,0.00,5350.37,2676.79,1.00
